In [1]:
%load_ext autoreload
%autoreload 2

import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

PosixPath('/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX')

In [13]:
import toponetx
from toponetx.transform.graph_to_simplicial_complex import graph_to_neighbor_complex
from topobenchmarkx.data.transforms.liftings import *
from topobenchmarkx.data.transforms.lifting import LiftingTransform
import networkx as nx

In [7]:
g = nx.Graph()
g.add_nodes_from([1, 2, 3, 4, 5, 6])
g.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (2, 5), (3, 4), (5, 6)])
print(g)

Graph with 6 nodes and 8 edges


In [8]:
simplicial = graph_to_neighbor_complex(g)
for ss in simplicial.simplices:
    print(ss)

frozenset({1})
frozenset({2})
frozenset({3})
frozenset({4})
frozenset({5})
frozenset({6})
frozenset({1, 2})
frozenset({1, 3})
frozenset({1, 4})
frozenset({2, 3})
frozenset({2, 4})
frozenset({3, 4})
frozenset({1, 5})
frozenset({2, 5})
frozenset({3, 5})
frozenset({4, 5})
frozenset({2, 6})
frozenset({5, 6})
frozenset({1, 2, 3})
frozenset({1, 2, 4})
frozenset({1, 3, 4})
frozenset({2, 3, 4})
frozenset({1, 2, 5})
frozenset({1, 3, 5})
frozenset({1, 4, 5})
frozenset({2, 3, 5})
frozenset({2, 4, 5})
frozenset({3, 4, 5})
frozenset({2, 5, 6})
frozenset({1, 2, 3, 4})
frozenset({1, 2, 3, 5})
frozenset({1, 2, 4, 5})
frozenset({1, 3, 4, 5})
frozenset({2, 3, 4, 5})
frozenset({1, 2, 3, 4, 5})


In [9]:
print(simplicial.up_laplacian_matrix(0).toarray())

[[ 4. -1. -1. -1. -1.  0.]
 [-1.  5. -1. -1. -1. -1.]
 [-1. -1.  4. -1. -1.  0.]
 [-1. -1. -1.  4. -1.  0.]
 [-1. -1. -1. -1.  5. -1.]
 [ 0. -1.  0.  0. -1.  2.]]


In [10]:
import torch_geometric
import torch

In [11]:
from itertools import combinations

In [35]:
data = torch_geometric.data.Data()
data.x = torch.zeros([6, 1])
data.edge_index = torch.tensor([[0, 0, 0, 1, 1, 1, 2, 4], [1, 2, 3, 2, 3, 4, 3, 5]])

In [36]:
lift = LiftingTransform("SimplicialNeighborhoodLifting", max_triangles=6)

In [37]:
lifted_data = lift(data)

In [38]:
print(lifted_data)

Data(x=[6, 1], edge_index=[2, 8], incidence_1=[6, 12], laplacian_down_1=[12, 12], laplacian_up_1=[6, 6], incidence_2=[12, 6], laplacian_down_2=[6, 6], laplacian_up_2=[12, 12])


In [34]:
lift.lifting(data)

Data(x=[6, 1], edge_index=[2, 8], incidence_1=[6, 12], laplacian_down_1=[12, 12], laplacian_up_1=[6, 6], incidence_2=[12, 6], laplacian_down_2=[6, 6], laplacian_up_2=[12, 12])

In [47]:
lift = SimplicialNeighborhoodLifting(max_triangles=6)

lifted_data = lift(data)

[(0,), (1,), (2,), (3,), (4,), (5,)]
[(0, 1), (2, 4), (1, 2), (0, 4), (3, 4), (1, 5), (0, 3), (1, 4), (2, 3), (0, 2), (4, 5), (1, 3)]
[(0, 2, 4), (0, 1, 2), (0, 1, 3), (1, 2, 3), (0, 1, 4), (1, 4, 5)]


In [50]:
print(lifted_data.laplacian_down_2.to_dense())

tensor([[ 3., -1.,  0.,  0., -1.,  0.],
        [-1.,  3., -1., -1., -1.,  0.],
        [ 0., -1.,  3., -1., -1.,  0.],
        [ 0., -1., -1.,  3.,  0.,  0.],
        [-1., -1., -1.,  0.,  3., -1.],
        [ 0.,  0.,  0.,  0., -1.,  3.]])


In [34]:
lift2 = CellCyclesLifting()

lifted_data = lift2(data)

In [35]:
print(lifted_data)

Data(x=[6, 1], edge_index=[2, 8], incidence_1=[6, 8], incidence_2=[8, 3], laplacian_up_1=[8, 8], laplacian_up_2=[3, 3], laplacian_down_2=[8, 8], laplacian_down_1=[6, 6], x_1=[8, 1])


In [36]:
from topobenchmarkx.data.transforms.liftings import SimplicialCliqueLifting

In [37]:
lift3 = SimplicialCliqueLifting()

lifted_data = lift3(data)

[(2,), (5,), (4,), (1,), (0,), (3,)]
[(1, 2), (5, 4), (0, 3), (1, 4), (2, 3), (0, 2), (1, 0), (1, 3)]
[(0, 2, 3), (1, 0, 2), (1, 0, 3), (1, 2, 3)]


In [40]:
print(lifted_data.laplacian_up_2.to_dense())

tensor([[ 2.,  0.,  0.,  0., -1., -1., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  2.,  0., -1., -1., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  0., -1.,  0.,  2., -1.,  0., -1.],
        [-1.,  0., -1.,  0., -1.,  2., -1.,  0.],
        [-1.,  0., -1.,  0.,  0., -1.,  2., -1.],
        [-1.,  0., -1.,  0., -1.,  0., -1.,  2.]])
